In [133]:
import random
from tqdm import tqdm
from math import exp
from math import log1p
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [134]:
def logloss(y_true, y_pred, eps=1e-15):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -(y_true * np.log(y_pred)).sum(axis=1).mean()

One instance

In [135]:
class Binary_Being():
    
    weights = []
    variables = []
    e = []
    eW = []
    ln = []
    lnW = []
    features = 0
    pruning = 1
    weight_sigma = 10
    variable_power = 3
    metric = 'acc'
    lr = 3
    
    def __init__(self, features=None, variable_power=3, weight_sigma=5, lr = 1, pruning=1, parents=[], metric='acc', random = True):
        if parents == []:
            if features==None:
                raise ValueError('features cannot be initialized as None')
            self.features = features
            self.variable_power = variable_power
            self.weight_sigma = weight_sigma
            self.lr = lr
            self.pruning = pruning
            self.variables = []
            self.weights = []
            self.metric = metric
            self.e = []
            self.ln = []
            self.eW = []
            self.lnW = []
            self.setup()
        else:
            self.features = parents[0].features
            self.variable_power = parents[0].variable_power
            self.weight_sigma = parents[0].weight_sigma
            self.lr = lr
            self.pruning = parents[0].pruning
            self.metric = parents[0].metric
            self.variables = []
            self.weights = []
            self.e = []
            self.ln = []
            self.eW = []
            self.lnW = []
            if random:
                self.setup()
            else:
                self.mutate(parents)
    
    def setup(self):
        for i in range(int(self.features*self.variable_power*random.random()*2+1)):
            temp = [random.randint(0, self.features-1)]
            for j in range(int(self.variable_power*random.random())):
                temp.append(random.randint(0, self.features-1))
            self.variables.append(temp)
        for i in range(len(self.variables)):
            if random.random() > .90:
                if random.random() > .5:
                    self.e.append(self.variables[i])
                else:
                    self.ln.append(self.variables[i])
        for i in range(len(self.variables)):
            self.weights.append(random.gauss(0, self.weight_sigma))
        for i in range(len(self.e)):
            self.eW.append(random.gauss(0, self.weight_sigma))
        for i in range(len(self.ln)):
            self.lnW.append(random.gauss(0, self.weight_sigma))

    
    def mutate(self, parents):
        for parent in parents:
            for var, weight in zip(parent.variables, parent.weights):
                if random.random() < self.pruning/len(parents):
                    self.variables.append(var)
                    self.weights.append(weight+random.gauss(0, self.lr))
            for e, eW in zip(parent.e, parent.eW):
                if random.random() < self.pruning/len(parents):
                    self.e.append(e)
                    self.eW.append(eW+random.gauss(0, self.lr))
            for ln, lnW in zip(parent.ln, parent.lnW):
                if random.random() < self.pruning/len(parents):
                    self.ln.append(ln)
                    self.lnW.append(lnW+random.gauss(0, self.lr))
        for i in range(int(self.pruning/random.random())):
            temp = [random.randint(0, self.features-1)]
            for j in range(int(self.variable_power*random.random())):
                temp.append(random.randint(0, self.features-1))
            if random.random() > .95:
                if random.random() > .5:
                    self.e.append(temp)
                    self.eW.append(random.gauss(0, self.weight_sigma))
                else:
                    self.ln.append(temp)
                    self.lnW.append(random.gauss(0, self.weight_sigma))
            self.variables.append(temp)
            self.weights.append(random.gauss(0, self.weight_sigma)) 
            
    
    def predict(self, x):
        pred = []
        for data in x:
            result = 0.0
            for var, weight in zip(self.variables, self.weights):
                mult = weight
                for elem in var:
                    mult *= data[elem]
                result += mult
            for e, eW in zip(self.e, self.eW):
                mult = 1
                for elem in e:
                    mult *= data[elem]
                if mult > 709:
                    result = 709
                result += eW*exp(mult) 
            for ln, lnW in zip(self.ln, self.lnW):
                mult = 1
                for elem in ln:
                    mult *= data[elem]
                result += lnW*log1p(abs(mult))
            try:
                act = 1 / (1 + exp(-result))
            except OverflowError:
                act = 0.0
            pred.append(act)
        return pred
    
    def evaluate(self, x, y, metric = ''):
        if metric == '': 
            metric = self.metric
        pred = self.predict(x)
        if metric=='acc':
            correct = 0
            incorrect = 0
            for y1, yp1 in zip(y, pred):
                if (yp1 >= .5 and y1 >= .5) or (yp1 < .5 and y1 < .5):
                    correct += 1
                else:
                    incorrect += 1
            return (correct+0.0)/(correct+incorrect+0.0)   
        elif metric == 'roc':
            pred = np.nan_to_num(pred)
            return roc_auc_score(y, pred)


In [136]:
a = Binary_Being(3)
a.predict(np.array([[1, 1, 1]]))

[0.9993710565424109]

In [137]:
class Categorical_Being():
    
    weights = []
    variables = []
    e = []
    eW = []
    ln = []
    lnW = []
    features = 0
    classes = 0
    pruning = 1
    weight_sigma = 10
    variable_power = 3
    metric = 'acc'
    lr = 3
    
    def __init__(self, features=None, classes=None, variable_power=3, weight_sigma=5, lr = 1, pruning=1, parents=[], metric='log_loss', random = True):
        if parents == []:
            if features==None:
                raise ValueError('features cannot be initialized as None')
            self.features = features
            self.classes = classes
            self.variable_power = variable_power
            self.weight_sigma = weight_sigma
            self.lr = lr
            self.pruning = pruning
            self.variables = []
            self.weights = []
            self.metric = metric
            self.e = []
            self.ln = []
            self.eW = []
            self.lnW = []
            self.setup()
        else:
            self.features = parents[0].features
            self.classes = parents[0].classes
            self.variable_power = parents[0].variable_power
            self.weight_sigma = parents[0].weight_sigma
            self.lr = lr
            self.pruning = parents[0].pruning
            self.metric = parents[0].metric
            self.variables = []
            self.weights = []
            self.e = []
            self.ln = []
            self.eW = []
            self.lnW = []
            if random:
                self.setup()
            else:
                self.mutate(parents)
    
    def setup(self):
        for j in range(self.classes):
            cv = []
            cvw = []
            ce = []
            cew = []
            cl = []
            clw = []
            for i in range(int(self.features*self.variable_power*random.random()*2+1)):
                temp = [random.randint(0, self.features-1)]
                for j in range(int(self.variable_power*random.random())):
                    temp.append(random.randint(0, self.features-1))
                cv.append(temp)
            for i in range(len(cv)):
                if random.random() > .95:
                    if random.random() > .5:
                        ce.append(cv[i])
                    else:
                        cl.append(cv[i])
            for i in range(len(cv)):
                cvw.append(random.gauss(0, self.weight_sigma))
            for i in range(len(ce)):
                cew.append(random.gauss(0, self.weight_sigma))
            for i in range(len(cl)):
                clw.append(random.gauss(0, self.weight_sigma))
            self.variables.append(cv)
            self.weights.append(cvw)
            self.e.append(ce)
            self.eW.append(cew)
            self.ln.append(cl)
            self.lnW.append(clw)

    
    def mutate(self, parents):
        for i in range(self.classes):
            for parent in parents:
                cv = []
                cvw = []
                ce = []
                cew = []
                cl = []
                clw = []
                for var, weight in zip(parent.variables[i], parent.weights[i]):
                    if random.random() < self.pruning/len(parents):
                        cv.append(var)
                        cvw.append(weight+random.gauss(0, self.lr))
                for e, eW in zip(parent.e[i], parent.eW[i]):
                    if random.random() < self.pruning/len(parents):
                        ce.append(e)
                        cew.append(eW+random.gauss(0, self.lr))
                for ln, lnW in zip(parent.ln[i], parent.lnW[i]):
                    if random.random() < self.pruning/len(parents):
                        cl.append(ln)
                        clw.append(lnW+random.gauss(0, self.lr))
            for j in range(int(self.pruning/random.random())):
                temp = [random.randint(0, self.features-1)]
                for q in range(int(self.variable_power*random.random())):
                    temp.append(random.randint(0, self.features-1))
                if random.random() > .99:
                    if random.random() > .5:
                        ce.append(temp)
                        cew.append(random.gauss(0, self.weight_sigma))
                    else:
                        cl.append(temp)
                        clw.append(random.gauss(0, self.weight_sigma))
                cv.append(temp)
                cvw.append(random.gauss(0, self.weight_sigma)) 
            self.variables.append(cv)
            self.weights.append(cvw)
            self.e.append(ce)
            self.eW.append(cew)
            self.ln.append(cl)
            self.lnW.append(clw)        
    
    def predict(self, x):
        pred = []
        for data in x:
            preAct = []
            for c in range(self.classes):
                result = 0.0
                for var, weight in zip(self.variables[c], self.weights[c]):
                    mult = weight
                    for elem in var:
                        mult *= data[elem]
                    result += mult
                for e, eW in zip(self.e[c], self.eW[c]):
                    mult = 1
                    for elem in e:
                        mult *= data[elem]
                    if mult > 709:
                        mult = 709
                    result += eW*exp(mult) 
                for ln, lnW in zip(self.ln[c], self.lnW[c]):
                    mult = 1
                    for elem in ln:
                        mult *= data[elem]
                    result += lnW*log1p(abs(mult))
                if result > 709:
                    result = 709
                preAct.append(result)
            preAct_exp = [exp(p) for p in preAct]
            sum_preAct = sum(preAct_exp)
            if sum_preAct == 0:
                sum_preAct = 1
            softmax = [pe/sum_preAct for pe in preAct_exp]
            pred.append(softmax)
        return pred

    
    def evaluate(self, x, y, metric = ''):
        if metric == '': 
            metric = self.metric
        pred = self.predict(x)
        pred = np.nan_to_num(pred)
        if metric=='log_loss':
            return 1/logloss(y, pred)
        elif metric=='acc':
            y = np.array(y)
            pred = np.array(pred)
            correct = 0
            incorrect = 0
            for y1, yp1 in zip(y, pred):
                if np.argmax(y1) == np.argmax(yp1):
                    correct += 1
                else:
                    incorrect += 1
            return (correct + 0.0)/(incorrect + correct + 0.0)


In [138]:
a = Categorical_Being(features=3, classes=3)
a.evaluate(np.array([[1, 1, 1], [2, 2, 2]]), np.array([[0, 0, 1], [1, 0, 0]]))

1.3184902246260441

Creating and Killing Instances

In [139]:
class Slaughterhouse():
    
    population = 0
    random = .3
    max_diversity = 20
    template = None
    decay = 1
    lr = 1
    beings = []
    parents = []
    outlast = 0.01
    min_diversity = 4
    being_type = None
    
    def __init__(self, template, population = 1000, random=0.4, max_diversity=20, decay=1, outlast=0.01, min_diversity = 4):
        self.population = population
        self.random = random
        self.max_diversity = max_diversity
        self.min_diversity = min_diversity
        self.template = template
        self.decay = decay
        self.lr = template.lr
        self.beings = []
        self.parents = []
        self.outlast = outlast
        self.being_type = type(template)
    
    def create_generation(self):
        for i in range(self.population):
            self.beings.append([0, self.being_type(parents=[self.template], lr=self.lr, random=True)])
    
    def score_generation(self, x, y):
        for i in tqdm(range(len(self.beings))):
            self.beings[i][0] = self.beings[i][1].evaluate(x, y)
        self.beings = sorted(self.beings, key=lambda x: x[0], reverse=True)
    
    def next_generation(self):
        self.get_parents()
        self.beings = []
        for parent in self.parents:
            self.beings.append([0, parent])
        for i in range(self.population - len(self.parents)):
            if random.random() > self.random:
                self.beings.append([0, self.being_type(parents=self.parents, lr=self.lr, random=False)])
            else:
                if random.random() > .5:
                    self.beings.append([0, self.being_type(parents=[self.parents[int(random.random()*len(self.parents))]], lr=self.lr, random=False)])
                else:
                    self.beings.append([0, self.being_type(parents=[self.template], lr=self.lr, random=True)])
    
    def get_parents(self):
        self.parents = []
        high = self.beings[0][0]
        for i in range(self.max_diversity):
            if high - self.outlast < self.beings[i][0] or i < self.min_diversity:
                self.parents.append(self.beings[i][1])
                print('Parent Acc:', self.beings[i][0])
    
    def go(self, x, y, generations=1):
        if self.beings == []:
            print('Start Generation')
            self.create_generation()
            self.score_generation(x, y)
        for i in range(generations):
            print('Generation ', i+1)
            self.next_generation()
            self.score_generation(x, y)
            self.lr *= self.decay
        self.get_parents()
        return self.parents
        

Using Titanic Data

In [140]:
titanic = pd.read_csv("titanic_dropped.csv")
x = np.array(titanic.drop(['Survived'], axis=1))
y = np.array(titanic['Survived'])

In [141]:
y2 = []
for elem in y:
    if elem == 1:
        y2.append([0, 1])
    else:
        y2.append([1, 0])
y2 = np.array(y2)

In [142]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y2, test_size=.2)

In [144]:
template = Categorical_Being(x.shape[1], classes=2, lr=1, variable_power=6, weight_sigma=5, metric='acc')

In [145]:
house = Slaughterhouse(template, population=1000, decay=.99)

In [146]:
final = house.go(x_train, y_train, generations=10)

Start Generation


100%|██████████| 1000/1000 [00:44<00:00, 22.35it/s]


Generation  1
Parent Acc: 0.7950963222416813
Parent Acc: 0.7933450087565674
Parent Acc: 0.7898423817863398
Parent Acc: 0.7863397548161121


100%|██████████| 1000/1000 [00:30<00:00, 33.15it/s]


Generation  2
Parent Acc: 0.8108581436077058
Parent Acc: 0.8073555166374781
Parent Acc: 0.8038528896672504
Parent Acc: 0.8021015761821366


...

Generation  9
Parent Acc: 0.8283712784588442
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8231173380035026
Parent Acc: 0.8231173380035026
Parent Acc: 0.8231173380035026
Parent Acc: 0.8231173380035026
Parent Acc: 0.8231173380035026
Parent Acc: 0.8231173380035026
Parent Acc: 0.8231173380035026
Parent Acc: 0.8231173380035026


100%|██████████| 1000/1000 [00:23<00:00, 42.49it/s]


Generation  10
Parent Acc: 0.8283712784588442
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164


100%|██████████| 1000/1000 [00:25<00:00, 39.98it/s]

Parent Acc: 0.830122591943958
Parent Acc: 0.8283712784588442
Parent Acc: 0.8283712784588442
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164


In [152]:
final[0].evaluate(x_test, y_test, metric = 'acc')

0.7272727272727273

In [150]:
final[0].variables, final[0].weights, final[0].e, final[0].eW, final[0].ln, final[0].lnW

([[[5, 6, 6],
   [5, 2],
   [0, 6, 0, 0, 6, 1],
   [4, 4],
   [3, 3, 5],
   [0, 4, 3, 0, 5],
   [1, 6, 2, 2],
   [1, 0, 6, 6, 1],
   [6, 3, 6, 3, 3, 5],
   [2, 0, 0, 1, 2, 2],
   [1],
   [4, 3, 2],
   [5, 6, 5, 6, 6],
   [2, 6, 4],
   [4, 1, 6, 1, 3],
   [4, 1, 6, 6],
   [4, 0],
   [2, 3],
   [1, 0, 2, 2, 2, 0],
   [5],
   [2, 2, 5],
   [2, 6, 1, 2, 3, 0],
   [6, 1],
   [5, 1, 6, 2],
   [4, 3, 2, 5],
   [3, 6, 5],
   [2, 6]],
  [[0, 2, 1, 1, 5, 6],
   [4, 4, 0, 1, 1, 5],
   [0],
   [5, 2],
   [1, 0, 4],
   [6],
   [2, 3, 4, 6, 5],
   [4, 1, 5, 3],
   [1, 6, 6, 0, 0, 2],
   [5, 4],
   [4, 6, 0, 6, 1],
   [0, 3, 1, 1, 3, 1],
   [2],
   [6, 0],
   [2, 6],
   [0, 0, 4, 4, 6, 6],
   [2, 1],
   [3, 2, 2],
   [6, 2, 5, 6, 3, 2],
   [4, 5],
   [3, 4],
   [1],
   [0, 4, 2, 5],
   [4, 0, 3],
   [2, 1, 6, 3, 5, 6],
   [3, 2, 2, 6, 1, 0],
   [2, 3, 5, 3, 3, 0],
   [1],
   [5, 2, 2, 0],
   [2, 6, 3, 1, 6],
   [3, 1, 2, 6, 2]]],
 [[-0.30748032128434966,
   -9.474330938589093,
   5.77824651649879,
  